# Welcome!

This notebook will allow you to customize prompts with different language models on your data.

# Install Prerequisite Libraries
The below code will depend on three Python "libraries" (software collections). Run the below cell once to install them.

In [ ]:
%pip install requests pandas scikit-learn

# Establish Your Working Directory

For our projects this semester we will upload a .csv file that has a "text" column. This will be our input to the language model.

First establish your working directory. Create a folder called "Jupyter" and put it in your Documents folder. Then run this cell.

In [1]:
# Import the libraries we need
from pathlib import Path  # This helps us work with file paths
import os                # This lets us change directories

def use_jupyter_folder():
    # Get the path to the Jupyter folder
    jupyter_folder = Path.home() / 'Documents' / 'Jupyter'
    
    # Try to change to that directory
    if jupyter_folder.exists():
        os.chdir(jupyter_folder)
        print(f"✅ Now using your Jupyter folder!")
        print(f"Current working directory: {Path.cwd()}")
    else:
        print("❌ Couldn't find the Jupyter folder in Documents.")
        print("Please make sure you've created it first.")

# Run this to switch to the Jupyter folder
use_jupyter_folder()

✅ Now using your Jupyter folder!
Current working directory: /Users/akpiper/Documents/Jupyter


# Upload Your Data

Next upload a .csv file of your choosing. Paste the filename where indicated at the bottom. This cell will output the column names.

In [2]:
import pandas as pd

def load_csv(filename):
   """Load CSV file and display info"""
   try:
       df = pd.read_csv(filename)
       print(f"✅ Successfully loaded {filename}")
       print(f"Shape: {df.shape[0]} rows, {df.shape[1]} columns")
       print("\nColumns in this dataset:")
       for col in df.columns:
           print(f"- {col}")
       return df
   except FileNotFoundError:
       print(f"❌ Could not find {filename} in {Path.cwd()}")
   except Exception as e:
       print(f"❌ Error loading file: {str(e)}")

########## PASTE YOUR .CSV FILENAME HERE ###########
df = load_csv("NarraDetect_Scalar.csv")


✅ Successfully loaded NarraDetect_Scalar.csv
Shape: 394 rows, 35 columns

Columns in this dataset:
- NARRATIVITY
- GENRE
- FILENAME
- TEXT
- AY_agency
- AY_event
- AY_world
- AY_overall
- ML_agency
- ML_event
- ML_world
- ML_overall
- LM_agency
- LM_event
- LM_world
- LM_overall
- avg_agency
- avg_event_seq
- avg_worldmaking
- avg_overall
- AvgDeviation
- Reader.Predicted.Label
- gpt.binary
- gpt.ordinal
- gpt.ordinal.mean
- svm.probability
- bert.probability
- longform.probability
- Llama.3.8B.ordinal
- Llama.3.8B.ordinal.mean
- Llama.3.8B.binary
- Llama.3.8B.fine.tuned.binary
- svm.binary
- bert.binary
- longform.binary


# Inspect Your Data

This cell will give you brief summary statistics on the input text column. This is the column you will use as part of your prompting.

In [3]:
#### DEFINE YOUR TEXT COLUMN HERE and NUMBER OF EXAMPLES YOU WANT TO SEE ######
def text_stats(df, text_column='TEXT', num_examples=2):
   """Display text statistics and examples"""
   # Calculate word counts
   word_counts = df[text_column].str.split().str.len()
   total_words = word_counts.sum()
   
   print(f"📊 Dataset Overview:")
   print(f"Total number of texts: {len(df)}")
   
   print(f"\n📝 Text Length Statistics:")
   print(f"Shortest text: {word_counts.min()} words")
   print(f"Longest text: {word_counts.max()} words")
   print(f"Average length: {word_counts.mean():.1f} words")
   print(f"Median length: {word_counts.median():.1f} words")
   print(f"Total words in dataset: {total_words:,} words")
   
   print(f"\n📚 Here are {num_examples} example texts from your data:")
   for i in range(num_examples):
       idx = df.index[i]
       text = df.loc[idx, text_column]
       length = len(text.split())
       print(f"Example {i+1}:")
       print(f"Length: {length} words")
       print(f"Text: {text}")

# Calculate statistics and show examples
text_stats(df)

📊 Dataset Overview:
Total number of texts: 394

📝 Text Length Statistics:
Shortest text: 16 words
Longest text: 444 words
Average length: 121.9 words
Median length: 111.0 words
Total words in dataset: 48,017 words

📚 Here are 2 example texts from your data:
Example 1:
Length: 187 words
Text: This was a history lesson under erasure that translated Stalinist mass violence into late-Soviet social discipline. A full explanation of Khrushchev’s speech and its aftermath and of the social processes surrounding the history and memory of Stalinist collective violence requires the articulation of novel theoretical positions. Analyses of legacies of mass historical violence in Europe and elsewhere typically cast the social arena as a battleground between memory and oblivion, projecting psychic mechanisms associated with the aftermath of trauma—most frequently those of repression and repetition compulsion—as metaphors for collective processes in which a society as a whole is conceived as repressin

# Ensure a Model is Loaded in Ollama

You will run this cell only once for the semester. Once the model is loaded you don't need to run it again.
But you do need to run it every time you want to test a new model.

In [4]:
model = "llama3:8b"  # Change this to your model name, e.g. "mistral", "codellama", etc.
#!ollama pull {model} >/dev/null 2>&1
print("Done!")

Done!


# Prompt Testing

In this cell you can choose a model from Ollama and then customize a prompt. You also need to specify the text column from your .csv. The cell chooses a random passage from the .csv and outputs the answer. You can run multiple times to keep testing answers on random passages.

In [5]:
import random
import random
import requests
import ast

##### INPUT YOUR PARAMETERS HERE #####
MODEL_NAME = "llama3:8b"  # Change model name here.
COLUMN_NAME = "TEXT"   # Change dataframe column name here
PROMPT_TEMPLATE = "Is this passage from a story? Answer only yes or no. {text}" #Change your prompt here
STRUCTURED = True
LABELS = ["Yes", "No"]

def query_ollama(text):
    """Query local ollama model with text"""
    url = "http://localhost:11434/api/generate"
    prompt = PROMPT_TEMPLATE.format(text=text)
    
    data = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "format": {
            "type": "object",
            "properties": {
                "label": {
                    "type": "string",
                    "enum" : LABELS
                },
            },
            "required": [
                    "label",
                ]
        } if STRUCTURED else ''
    }
    
    try:
        # Check if model exists
        model_url = "http://localhost:11434/api/tags"
        models = requests.get(model_url).json()
        available_models = [model['name'] for model in models['models']]
        
        if MODEL_NAME not in available_models:
            print(f"❌ Model '{MODEL_NAME}' not found.")
            print(f"Available models: {', '.join(available_models)}")
            print(f"\nTo install {MODEL_NAME}, run this in terminal:")
            print(f"ollama pull {MODEL_NAME}")
            return None

        response = requests.post(url, json=data)
        if response.status_code == 404:
            print("❌ Ollama service not running.")
            print("Start ollama by running 'ollama serve' in terminal")
            return None

        result = response.json()
        if STRUCTURED:
            return ast.literal_eval(result['response'])['label']
        return result['response']

    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama")
        print("1. Check if Ollama is installed") 
        print("2. Start Ollama by running 'ollama serve' in terminal")
        return None
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

def analyze_random_text(df):
  """Analyze a random text from dataset"""
  random_idx = random.randint(0, len(df)-1)
  text = df.iloc[random_idx][COLUMN_NAME]
  print("\n📖 SAMPLE PASSAGE:")
  print(text)
  print("\n🤖 MODEL RESPONSE:")
  return query_ollama(text)

# Run
result = analyze_random_text(df)
if result:
    print(result)


📖 SAMPLE PASSAGE:
After his mother rescues him, he joins his swarm of siblings flying above and is bumped and jostled. Skinny even by mosquito standards, and fearing he will end up doused again, Dinnn decides he would rather walk than fly.,This decision inevitably leads to Dinnn’s being taunted by older mosquitos and largely forgotten by his family. The ridicule intensifies when Dinnn finds a tiny, discarded motorcycle jacket (with “Wasp Brothers” inscribed on the back), which he takes to wearing over his unused wings.,Dinnn is put into a situation that forces him to ditch his jacket and take flight, which allows him to tag along with the rest of his family as they hitch a ride in a minivan to the “wild,” where his mother came from, and where his half-brother still resides.,The transition from the first part of the narrative to the latter sections (after Dinnn learns to fly) is less smooth than it could be. The young bug’s fear of flying is suddenly and easily resolved, and lends a no

# Sample your data

In this cell you will downsample your .csv file to run a mini test in class. For your final report you will run the model(s) against all rows (or a minimum sample of 100). This function allows you to determine the number of rows you sample and stores the new table. Change the variable "n" for number of rows to sample. 

** Note: every time you run this cell you will get a new random sample.

In [6]:
def sample_texts(df, n=20): # change n here to adjust the size of your sample
    """
    Sample n random rows from the dataset
    
    Parameters:
    df (pandas.DataFrame): Your dataset
    n (int): Number of samples to take
    """
    global sample_df
    sample_df = df.sample(n=n)
    
# Create sample with 3 rows
sample_texts(df)

# Run your prompt on your sample data

In this cell you will run your prompt on the sampled data from above. The outputs will be stored as a new column named after the model you are using. In the next cell you can view those results. The cell will output "Completed" when complete.

In [7]:
import requests

##### INPUT YOUR PARAMETERS HERE #####
MODEL_NAME = "llama3:8b"  # Change model name here.
COLUMN_NAME = "TEXT"   # Change dataframe column name here
PROMPT_TEMPLATE = "Is this passage from a story? Answer only yes or no. {text}"
STRUCTURED = True
LABELS = ["Yes", "No"]

def query_ollama(text):
    """Query local ollama model with text"""
    url = "http://localhost:11434/api/generate"
    prompt = PROMPT_TEMPLATE.format(text=text)
    
    data = {
        "model": MODEL_NAME,
        "prompt": prompt,
        "stream": False,
        "format": {
            "type": "object",
            "properties": {
                "label": {
                    "type": "string",
                    "enum" : LABELS
                },
            },
            "required": [
                    "label",
            ]
        } if STRUCTURED else ''
    }
    
    try:
        # Check if model exists
        model_url = "http://localhost:11434/api/tags"
        models = requests.get(model_url).json()
        available_models = [model['name'] for model in models['models']]
        
        if MODEL_NAME not in available_models:
            print(f"❌ Model '{MODEL_NAME}' not found.")
            return None

        response = requests.post(url, json=data)
        if response.status_code == 404:
            print("❌ Ollama service not running.")
            return None

        result = response.json()
        if STRUCTURED:
            return ast.literal_eval(result['response'])['label']
        return result['response']
    except requests.exceptions.ConnectionError:
        print("❌ Cannot connect to Ollama")
        return None
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        return None

def analyze_all_texts(df):
    """Analyze all texts in the dataframe"""
    # Create new column for responses using model name
    df[MODEL_NAME] = df[COLUMN_NAME].apply(query_ollama)
    return df

# Run analysis on all rows
sample_df = analyze_all_texts(sample_df)
print("Completed!")

Completed!


# Inspect your outputs

You can quickly scan your results by printing out the first N examples. Change the final integer to print more or less. Shows the passage + prompt output.

In [8]:
print(sample_df[[COLUMN_NAME, MODEL_NAME]].head(5))

                                                  TEXT llama3:8b
79   It just means white man,” says Kyle when Jared...       Yes
267  That's a very different thing. But when it com...       Yes
29   Purpose - The aim of this paper is to examine ...       Yes
107  Resolving to dance at an upcoming powwow, John...       Yes
2    Uli painting does not furnish indigenous subje...       Yes


Print a single passage by row number.

In [9]:
# Display a specific row (change row_number to view different rows)
row_number = 3  # Change this number to view different rows
print(f"\nDetailed view of row {row_number}:")
print(f"\nTEXT:\n{sample_df[COLUMN_NAME].iloc[row_number]}")
print(f"\n{MODEL_NAME} response:\n{sample_df[MODEL_NAME].iloc[row_number]}")


Detailed view of row 3:

TEXT:
Resolving to dance at an upcoming powwow, John finds a way to stand up for himself so he can keep dancing and playing soccer.,John is an appealing character, a passionate teenager who works hard at both his prevailing interests. While he becomes a great dancer a little too quickly, his initial struggles are entertaining and realistic. Scenes between him and his parents and energetic younger sister, Jen, are especially well drawn, and the book’s supportive family dynamic contrasts well with the racism and conflict mentioned elsewhere. John’s relationships with his friends, however, are lacking, and more space could have been devoted to his friendship with soccer teammate Aiden, who is understanding of John’s choices but is an otherwise flat character.,While the book is not overly focused on racism, passing mention of some of John and Jen’s negative experiences, and acknowledgement of topics like cultural appropriation, help ground the story without feelin

# Compare your outputs to another reference column

In the following cells you will compare the accuracy of your outputs to already annotated data. First you need to identify the "reference" column. These are the annotations. Second, you need to align your outputs with those of the reference column. Typically these will consist of a few number of codes. So the first step is finding out these codes so you can align them with your outputs.

## What are the annotation categories of my data

Output a table of the categories and their counts in your data. Change the reference column name accordingly.

In [10]:
###### Get value counts of the reference column. 
###### CHANGE reference column name based on yoru data. 
######It comes after df["reference_name"].value_counts()
reference_counts = sample_df["Reader.Predicted.Label"].value_counts()

print("Categories and their counts:")
for category, count in reference_counts.items():
    print(f"{category}: {count}")
print(f"\nTotal samples: {len(sample_df)}")

Categories and their counts:
0: 13
1: 7

Total samples: 20


Do the same thing for your model outputs.

In [11]:
# Get value counts of the model responses
model_counts = sample_df[MODEL_NAME].value_counts()

print(f"Response categories from {MODEL_NAME}:")
for category, count in model_counts.items():
    print(f"{category}: {count}")
print(f"\nTotal samples: {len(sample_df)}")

Response categories from llama3:8b:
Yes: 17
No: 3

Total samples: 20


## Clean your model outputs

In order to align your outputs with the reference column you first need to standardize your outputs. This cell outputs all of the types of response you got.

In [12]:
import re

def clean_responses(df, model_column=MODEL_NAME):
    """
    Clean and standardize model responses to 'Yes' or 'No' using case-insensitive pattern matching
    """
    def standardize_response(response):
        # Case insensitive search for 'yes' or 'no'
        if re.search(r'yes', response, re.IGNORECASE):
            return 'Yes'
        elif re.search(r'no', response, re.IGNORECASE):
            return 'No'
        else:
            print(f"Warning: Unexpected response format: '{response}'")
            return 'Unknown'
            
    # Create new cleaned column
    cleaned_column = f"{model_column}_cleaned"
    df[cleaned_column] = df[model_column].apply(standardize_response)
    return df

# Clean the responses
sample_df = clean_responses(sample_df)

# Show the new counts
cleaned_counts = sample_df[f"{MODEL_NAME}_cleaned"].value_counts()
print(f"Cleaned response categories:")
for category, count in cleaned_counts.items():
    print(f"{category}: {count}")
print(f"\nTotal samples: {len(sample_df)}")

Cleaned response categories:
Yes: 17
No: 3

Total samples: 20


## Align outputs with reference column

Next transform the reference column categories to match your model outputs. This is going to be a custom job every time. Paste the current code into Claude or GPT and ask for help changing the code based on your situation. Give the AI as much information as possible to help you and then paste the code back in here.


In [13]:
def transform_reference_labels(df):
    """
    Transform reference labels from 1/0 to Yes/No
    """
    def map_label(value):
        if value == 1:
            return 'Yes'
        elif value == 0:
            return 'No'
        else:
            print(f"Warning: Unexpected reference value: {value}")
            return 'Unknown'
            
    # Create new transformed column
    reference_cleaned = "Reader.Predicted.Label_cleaned"
    df[reference_cleaned] = df["Reader.Predicted.Label"].apply(map_label)
    return df

# Transform reference labels
sample_df = transform_reference_labels(sample_df)

# Show the new counts
reference_counts = sample_df["Reader.Predicted.Label_cleaned"].value_counts()
print(f"Reference label categories:")
for category, count in reference_counts.items():
    print(f"{category}: {count}")
print(f"\nTotal samples: {len(sample_df)}")

Reference label categories:
No: 13
Yes: 7

Total samples: 20


# Calculate Precision, Recall, and F1 Score

These are measures of agreement we will use this semester to see how well a model + prompt performs. Make sure to adjust the column names below to match your goals.

In [14]:
from sklearn.metrics import precision_score, recall_score, f1_score

def calculate_metrics(df, model_column=MODEL_NAME):
    """
    Calculate precision, recall, and F1 score for model predictions
    """
    ##### CHANGE COLUMN NAMES ######
    y_true = df['Reader.Predicted.Label_cleaned'] ### This column is the original annotated data that has been transformed.
    y_pred = df[f'{model_column}_cleaned'] ### This is your data that has been cleaned.
    
    # Convert to binary format for sklearn (Yes -> 1, No -> 0)
    y_true_binary = (y_true == 'Yes').astype(int)
    y_pred_binary = (y_pred == 'Yes').astype(int)
    
    # Calculate metrics
    precision = precision_score(y_true_binary, y_pred_binary)
    recall = recall_score(y_true_binary, y_pred_binary)
    f1 = f1_score(y_true_binary, y_pred_binary)
    
    # Print results
    print(f"Metrics for {model_column}:")
    print(f"Precision: {precision:.3f}")
    print(f"Recall: {recall:.3f}")
    print(f"F1 Score: {f1:.3f}")
    
# Calculate metrics
calculate_metrics(sample_df)

Metrics for llama3:8b:
Precision: 0.412
Recall: 1.000
F1 Score: 0.583


## Output a confusion matrix

In [15]:
from sklearn.metrics import confusion_matrix
import pandas as pd

def display_confusion_matrix(df, model_column=MODEL_NAME):
    """
    Create and display a confusion matrix comparing model predictions to reference labels
    """
    # Get the cleaned columns
    y_true = df['Reader.Predicted.Label_cleaned']
    y_pred = df[f'{model_column}_cleaned']
    
    # Create confusion matrix
    cm = confusion_matrix(y_true, y_pred, labels=['Yes', 'No'])
    
    # Convert to pandas DataFrame for better display
    cm_df = pd.DataFrame(
        cm, 
        index=['True Yes', 'True No'],
        columns=['Predicted Yes', 'Predicted No']
    )
    
    print("Confusion Matrix:")
    print(f"Model: {model_column}")
    print(cm_df)
    print("\nReading the matrix:")
    print(f"True Positives (Correct Yes): {cm[0,0]}")
    print(f"False Negatives (Missed Yes): {cm[0,1]}")
    print(f"False Positives (Wrong Yes): {cm[1,0]}")
    print(f"True Negatives (Correct No): {cm[1,1]}")

# Display confusion matrix
display_confusion_matrix(sample_df)

Confusion Matrix:
Model: llama3:8b
          Predicted Yes  Predicted No
True Yes              7             0
True No              10             3

Reading the matrix:
True Positives (Correct Yes): 7
False Negatives (Missed Yes): 0
False Positives (Wrong Yes): 10
True Negatives (Correct No): 3


## Inspect errors

Your errors can take the form of false positives (e.g. when the model thinks a passage is a story but isn't) or false negatives (e.g. when your model thinks the passage isn't a story but is).

In [16]:
def show_error_examples(df, model_column=MODEL_NAME):
    """
    Display examples of false positives and false negatives with their corresponding text passages
    """
    # Get relevant columns
    ref_col = 'Reader.Predicted.Label_cleaned'
    pred_col = f'{model_column}_cleaned'
    
    # Find false positives (model predicted Yes when true label was No)
    false_positives = df[
        (df[pred_col] == 'Yes') & 
        (df[ref_col] == 'No')
    ]
    
    # Find false negatives (model predicted No when true label was Yes)
    false_negatives = df[
        (df[pred_col] == 'No') & 
        (df[ref_col] == 'Yes')
    ]
    
    # Display one example of each if available
    print("=== FALSE POSITIVE EXAMPLE ===")
    print("(Model incorrectly predicted Yes when the true label was No)")
    if len(false_positives) > 0:
        fp_example = false_positives.sample(1).iloc[0]
        print(f"\nPassage:")
        print(fp_example['TEXT'])
        print(f"\nModel response:")
        print(fp_example[model_column])
    else:
        print("No false positives found!")
        
    print("\n=== FALSE NEGATIVE EXAMPLE ===")
    print("(Model incorrectly predicted No when the true label was Yes)")
    if len(false_negatives) > 0:
        fn_example = false_negatives.sample(1).iloc[0]
        print(f"\nPassage:")
        print(fn_example['TEXT'])
        print(f"\nModel response:")
        print(fn_example[model_column])
    else:
        print("No false negatives found!")

# Show error examples
show_error_examples(sample_df)

=== FALSE POSITIVE EXAMPLE ===
(Model incorrectly predicted Yes when the true label was No)

Passage:
What makes you call real life confusion is that it presents, as if they were dissolved in one another, a lot of differents which conception breaks life's flow by keeping apart. But _are_ not differents actually dissolved in one another? Hasn't every bit of experience its quality, its duration, its extension, its intensity, its urgency, its clearness, and many aspects besides, no one of which can exist in the isolation in which our verbalized logic keeps it? They exist only _durcheinander_. Reality always is, in M. Bergson's phrase, an endosmosis or conflux of the same with the different: they compenetrate and telescope.

Model response:
Yes

=== FALSE NEGATIVE EXAMPLE ===
(Model incorrectly predicted No when the true label was Yes)
No false negatives found!
